# Instructlab local - 02 Test inference

## 1. VLLM inference

In [2]:
models = {
"mistral":"mistralai/Mistral-7B-v0.3",
"mistral-instruct":"mistralai/Mistral-7B-Instruct-v0.3",
"llama3":"meta-llama/Meta-Llama-3-8B",
"llama3-instruct":"meta-llama/Meta-Llama-3-8B-Instruct",
"phi3-mini":"microsoft/Phi-3-mini-4k-instruct",
"phi3-small":"microsoft/Phi-3-small-8k-instruct",
"mixtral-q3":"mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-3bit-metaoffload-HQQ",
"mixtral-q2":"mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-2bit_g16_s128-HQQ"
}

**IMPORTANT: always set the local download cache directory explicitly**

In [3]:
DOWNLOAD_CACHE_DIR = "/models/huggingface/transformers"

from pathlib import Path
print(f"Download cache dir: {DOWNLOAD_CACHE_DIR} {('OK' if Path(DOWNLOAD_CACHE_DIR).exists() else 'KO')}")

Download cache dir: /models/huggingface/transformers OK


### 1.1 Mistral 7B instruct v0.3

In [4]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = models["mistral-instruct"]

print(f"Loading model {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model_name, kv_cache_dtype="fp8", gpu_memory_utilization=0.99, download_dir=DOWNLOAD_CACHE_DIR)

/workspace/instructlab-local/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading model mistralai/Mistral-7B-Instruct-v0.3
INFO 06-01 21:46:47 config.py:390] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile, it may cause accuracy drop without a proper scaling factor
INFO 06-01 21:46:47 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='mistralai/Mistral-7B-Instruct-v0.3', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir='/models/huggingface/transformers', load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=fp8, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=mistralai/Mistral-7B-Instr

/workspace/instructlab-local/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


WARNING 06-01 21:46:48 utils.py:451] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 06-01 21:46:48 selector.py:130] Cannot use FlashAttention-2 backend for FP8 KV cache.
INFO 06-01 21:46:48 selector.py:51] Using XFormers backend.


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


INFO 06-01 21:46:49 selector.py:130] Cannot use FlashAttention-2 backend for FP8 KV cache.
INFO 06-01 21:46:49 selector.py:51] Using XFormers backend.
INFO 06-01 21:46:49 weight_utils.py:207] Using model weights format ['*.safetensors']
INFO 06-01 21:46:52 model_runner.py:146] Loading model weights took 13.5083 GB
INFO 06-01 21:46:55 gpu_executor.py:83] # GPU blocks: 6082, # CPU blocks: 4096
INFO 06-01 21:46:55 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-01 21:46:55 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-01 21:46:59 model_runner.py:924] Graph capturing finished in 4 secs.


In [ ]:
pip install mistral_common==1.1.0

In [5]:
from transformers.utils.hub import cached_file 

tokenizer_model_file = cached_file(model_name, "tokenizer.model.v3", cache_dir=DOWNLOAD_CACHE_DIR)
tokenizer_model_file

'/models/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/83e9aa141f2e28c82232fea5325f54edf17c43de/tokenizer.model.v3'

In [6]:
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

tokenizer = MistralTokenizer.from_file(tokenizer_model_file)

In [7]:
from mistral_common.protocol.instruct.messages import SystemMessage, UserMessage, AssistantMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

completion_request = ChatCompletionRequest(messages=[UserMessage(content="quelles sont les principales ouvertures des échecs ?")])

tokens = tokenizer.encode_chat_completion(completion_request)
tokens.text

'<s>[INST]▁quelles▁sont▁les▁principales▁ouvertures▁des▁échecs▁?[/INST]'

In [8]:
from typing import Optional
from vllm.lora.request import LoRARequest

def vllm_stream_output(
    llm: LLM,
    prompt: str,
    sampling_params: Optional[SamplingParams] = None,
    lora_request: Optional[LoRARequest] = None
    ):

    if sampling_params is None:
        # Use default sampling params.
        sampling_params = SamplingParams()

    llm._validate_and_add_requests(
        inputs=prompt,
        params=sampling_params,
        lora_request=lora_request,
    )

    while llm.llm_engine.has_unfinished_requests():
        step_outputs = llm.llm_engine.step()
        output = step_outputs[0]
        yield output.outputs[0].text
        if output.finished:
            yield output

def vllm_display_output(
    llm: LLM,
    prompt: str,
    sampling_params: Optional[SamplingParams] = None,
    lora_request: Optional[LoRARequest] = None
    ):
          
    last_length = 0
    for result in vllm_stream_output(llm, prompt, sampling_params=sampling_params, lora_request=lora_request):
        if isinstance(result,str):
            print(result[last_length:], end="", flush=True)
            last_length = len(result)
        else:
            print()
            print("--------------------------")
            input_tokens = len(result.prompt_token_ids)
            output_tokens = len(result.outputs[0].token_ids)
            total_time = result.metrics.finished_time-result.metrics.arrival_time
            time_to_first_token = result.metrics.first_token_time-result.metrics.arrival_time
            tokens_per_sec = (output_tokens-1)/(total_time-time_to_first_token)
            print(f"{output_tokens} tokens generated in {total_time:.2f} sec | prompt: {input_tokens:.2f} tokens | time to first token: {time_to_first_token:.2f} sec | throughput: {tokens_per_sec:.2f} tokens/sec")

In [9]:
max_tokens = 1024
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=max_tokens)

vllm_display_output(llm, tokens.text,sampling_params=sampling_params)

 Les principales ouvertures d'échecs, classées par ordre de popularité, sont :

1. Ouverture française (ECO C00) : 1.e4 e6 2.d4 d5
2. Défense sicilienne (ECO B00-B99) : 1.e4 c5
3. Ouverture du roi (ECO A00-A99) : 1.d4
4. Défense indienne (ECO E00-E99) : 1.d4 Cf6
5. Défense gréco (ECO D00-D99) : 1.e4 e5
6. Défense caro-kann (ECO B10-B99) : 1.e4 c6
7. Ouverture scandinave (ECO A10-A99) : 1.e4 d5 2.exd5 Cf6
8. Ouverture espagnole (ECO C80-C99) : 1.e4 e5 2.Cf3 Cc6
9. Défense sicilienne variante Najdorf (ECO B97) : 1.e4 c5 2.d4 cxd4 3.Cc3 a6 4.Cf3 Cf6 5.dxc5
10. Défense sicilienne variante Scheveningen (ECO B84) : 1.e4 c5 2.Nf3 d6 3.d4 cxd4 4.Nxd4 Nf6 5.Nc3 a6 6.f3.
--------------------------
403 tokens generated in 7.75 sec | prompt: 18.00 tokens | time to first token: 0.27 sec | throughput: 53.72 tokens/sec


In [11]:
requests_results = llm.generate(tokens.text, sampling_params=sampling_params)
result = requests_results[0]
output = result.outputs[0]
print(output.text)
print("-------------------------")
print(f"{len(output.token_ids)} tokens generated in {result.metrics.finished_time-result.metrics.arrival_time:.2f} sec")

Processed prompts: 100%|██████████| 1/1 [00:18<00:00, 18.70s/it, Generation Speed: 54.77 toks/s]

 Les principales ouvertures d'échecs sont les suivantes :

1. Ouverture française (ECO C00-C99) : 1. e4 e6
2. Ouverture anglaise (ECO A00-A99) : 1. c4
3. Défense sicilienne (ECO B00-B99) : 1. e4 c5
4. Défense indienne (ECO E00-E99) : 1. d4
5. Défense sicilienne, variante Najdorf (ECO B97) : 1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Cf6 5. Nc3 a6 6. Rd1
6. Défense sicilienne, variante Scheveningen (ECO B84) : 1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Be2
7. Ouverture du roi (ECO D00-D99) : 1. d4 d5
8. Défense russe (ECO E10-E99) : 1. d4 d5 2. c4 c6
9. Défense gréco (ECO D05-D09) : 1. d4 d5 2. c4 c6 3. Cf3
10. Défense caro-kann (ECO B10-B99) : 1. e4 c6 2. d4 d5
11. Défense benoni (ECO A60-A69) : 1. d4 c5 2. d5 e6 3. c4
12. Défense scandinave (ECO B03) : 1. e4 d5 2. exd5 Nf6 3. Nc3
13. Défense danoise (ECO B04) : 1. e4 e5 2. d4 exd4 3. Qxd4 Nc6
14. Défense sicilienne, variante Taimanov (ECO B88) : 1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Bd3
15. Défense sicilienne, 

In [10]:
import os

gradio_server_port = int(os.environ.get('GRADIO_PORT'))
gradio_root_path = os.environ.get('GRADIO_BASE_URL')

print(f"This port used by gradio apps in the container: {gradio_server_port}")
print(f"Will be exposed at the following url on the virtual machine: {gradio_root_path}")

from IPython.display import display, HTML

display(HTML(f'<a href="{gradio_root_path}/" target="_blank">Click here to access the gradio app</a>'))

This port used by gradio apps in the container: 7860
Will be exposed at the following url on the virtual machine: /notebooks/gradio


In [11]:
import gradio as gr

system_prompt = "Tu es MonIA, un assistant expert qui essaie toujours d'être concis, exact, utile et poli."

def apply_chat_template(system_prompt, history, message):
    messages = [SystemMessage(content=system_prompt)]
    for turn in history:
        messages.append(UserMessage(content=turn[0]))
        messages.append(AssistantMessage(content=turn[1]))
    messages.append(UserMessage(content=message))
    completion_request = ChatCompletionRequest(messages=messages)
    tokens = tokenizer.encode_chat_completion(completion_request)
    return tokens.text

def process_message(message, history, system_prompt):
    prompt = apply_chat_template(system_prompt, history, message)
    for result in vllm_stream_output(llm, prompt, sampling_params=sampling_params):
        if isinstance(result,str):
            yield result
        else:
            print()
            print("--------------------------")
            input_tokens = len(result.prompt_token_ids)
            output_tokens = len(result.outputs[0].token_ids)
            total_time = result.metrics.finished_time-result.metrics.arrival_time
            time_to_first_token = result.metrics.first_token_time-result.metrics.arrival_time
            tokens_per_sec = (output_tokens-1)/(total_time-time_to_first_token)
            print(f"{output_tokens} tokens generated in {total_time:.2f} sec | prompt: {input_tokens:.2f} tokens | time to first token: {time_to_first_token:.2f} sec | throughput: {tokens_per_sec:.2f} tokens/sec")


demo = gr.ChatInterface(process_message, title=model_name, additional_inputs=[
        gr.Textbox(system_prompt, label="System Prompt")
    ], additional_inputs_accordion="Configuration").queue()
demo.launch(inline=False, share=False, server_name="0.0.0.0", server_port=gradio_server_port, root_path=gradio_root_path)

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.



--------------------------
69 tokens generated in 1.58 sec | prompt: 49.00 tokens | time to first token: 0.34 sec | throughput: 54.76 tokens/sec

--------------------------
121 tokens generated in 2.47 sec | prompt: 129.00 tokens | time to first token: 0.31 sec | throughput: 55.61 tokens/sec

--------------------------
195 tokens generated in 3.74 sec | prompt: 267.00 tokens | time to first token: 0.23 sec | throughput: 55.21 tokens/sec

--------------------------
195 tokens generated in 3.95 sec | prompt: 277.00 tokens | time to first token: 0.33 sec | throughput: 53.73 tokens/sec


In [34]:
demo.close()

Closing server running on port: 7860


In [ ]:
import time
prompt = "Quels sont les avantages du Crédit Mutuel ?"

# System prompt
messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": prompt}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Banque populaire ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Caise d'épargne ?"}
]
]

# Generate outputs
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

start_time = time.time()  # Record the start time
outputs = llm.generate(text, sampling_params)
end_time = time.time()  # Record the end time
    
# Print the outputs.
tokenscount = 0
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    tokenscount = tokenscount + len(output.outputs[0].token_ids)
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    
print(f"Performance: {int(tokenscount/(end_time-start_time))} tokens/sec")